In [ ]:
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
titanic = sns.load_dataset("titanic").dropna(subset=["age", "sex", "fare", "class", "embarked", "alone"])
X = titanic[["age", "fare", "sex", "class", "embarked", "alone"]]
y = titanic["survived"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [ ]:
numerical_features = ["age", "fare"]
categorical_features = ["sex", "class", "embarked", "alone"]

preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numerical_features),
        ("cat", OneHotEncoder(drop="first"), categorical_features)
    ]
)

In [ ]:
pipe = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", LogisticRegression(max_iter=2000, class_weight="balanced"))
])

param_grid = {
    "model__C": [0.01, 0.1, 1, 10, 100],
    "model__solver": ["lbfgs", "saga"]
}

grid = GridSearchCV(
    pipe, 
    param_grid, 
    cv=5, 
    scoring="accuracy", 
    n_jobs=-1
)

grid.fit(X_train, y_train)

In [ ]:
y_pred = grid.predict(X_test)

print("Best Parameters:", grid.best_params_)
print("Train Accuracy:", grid.best_score_)
print("Test Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


Best Parameters: {'model__C': 10, 'model__solver': 'lbfgs'}
Train Accuracy: 0.7873001086787765
Test Accuracy: 0.7692307692307693

Classification Report:
               precision    recall  f1-score   support

           0       0.84      0.75      0.80        85
           1       0.69      0.79      0.74        58

    accuracy                           0.77       143
   macro avg       0.76      0.77      0.77       143
weighted avg       0.78      0.77      0.77       143

Confusion Matrix:
 [[64 21]
 [12 46]]


## END